In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from dataclasses import dataclass

@dataclass
class MetOceanParameters:
    Hm0: float        # Significant wave height [m]
    Tp: float         # Peak wave period [s]
    T02: float        # Mean zero-crossing period [s]
    CS_total: float   # Depth-averaged current speed [m/s]
    WS160: float      # Wind speed at 160 m [m/s]
    WS160_adj: float  # Wind speed at 160 m (+7%) [m/s]
    WL_tot: float     # Total water level [m MSL]
    Hmax: float       # Maximum wave height [m]
    T_Hmax: float     # Wave period associated with Hmax [s]
    Cmax_SWL: float   # Max crest level w.r.t. SWL [m]
    Cmax_MSL: float   # Max crest level w.r.t. MSL [m]


# -------------------------
# 200-year return period
# -------------------------

HWL = MetOceanParameters(
    Hm0=9.9,
    Tp=15.3,
    T02=9.8,
    CS_total=0.5,
    WS160=29.7,
    WS160_adj=31.8,
    WL_tot=1.3,
    Hmax=19.2,
    T_Hmax=12.1,
    Cmax_SWL=12.8,
    Cmax_MSL=14.6
)

LWL = MetOceanParameters(
    Hm0=8.7,
    Tp=13.4,
    T02=8.5,
    CS_total=0.5,
    WS160=32.9,
    WS160_adj=35.2,
    WL_tot=-0.2,
    Hmax=16.6,
    T_Hmax=10.5,
    Cmax_SWL=11.0,
    Cmax_MSL=10.7
)


## Parameters

In [7]:
# General constants
g = 9.8 # m/s^2
rho = 1025 # kg/m^3

## Combined current & waves

In [8]:
def soulsby_um(H_m0, T_p, h, g=9.81):
    """
    Compute U_m,Soulsby using the Soulsby formula.

    Parameters
    ----------
    H_m0 : float or ndarray
        Significant wave height (m)
    T_p : float or ndarray
        Peak wave period (s)
    h : float or ndarray
        Water depth (m)
    g : float, optional
        Gravitational acceleration (m/s^2), default 9.81

    Returns
    -------
    U_m : float or ndarray
        Maximum orbital velocity (m/s)
    """
    term1 = H_m0 / (2 * np.sqrt(2))
    term2 = np.sqrt(g / h)
    exponent = -((4.745 / T_p) * np.sqrt(h / g))**2.1

    U_m = term1 * term2 * np.exp(exponent)
    return U_m

Bed orbital velocity (U_m): 1.050232451074436 m/s


In [10]:
def relative_velocity(u_c, U_w_bed):
    """
    Compute relative velocity and interpret flow regime (scalar version).
    """
    U_rel = u_c / (u_c + U_w_bed)

    if np.isclose(U_rel, 0.0):
        regime = "waves-only"
    elif np.isclose(U_rel, 1.0):
        regime = "current-only"
    else:
        regime = "combined current and waves"

    return U_rel, regime

In [11]:
H_m0  = 2.5 # m
T_p   = 8.0 # s
h     = 5.0 # m

bed_um = soulsby_um(H_m0, T_p, h)
print(f"Bed orbital velocity (U_m): {bed_um} m/s")

Bed orbital velocity (U_m): 1.050232451074436 m/s


## Scour depth